In [20]:
import collections
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

SPAM_col = collections.Counter()
NOT_SPAM_col = collections.Counter()
P_SPAM = collections.Counter()
P_not_SPAM = collections.Counter()
SPAM_count = 0 #накопелнное количество писем спама для обучения
NOT_SPAM_count = 0 #накопленное количество писем неспама для обучения
P_All_SPAM = 0.5 #вероятность спама писемь, начальное значение
SPAM_word_count = 0 # накопелнное количество слов среди писем спама для обучения
NOT_SPAM_word_count = 0 # накопелнное количество слов среди писем неспама для обучения
SPAM = 0
NOT_SPAM = 1
SPAM_DICT = {SPAM: "SPAM", NOT_SPAM: "NOT SPAM"}
min_word_len = 9 #минимальная длинна слова которое нужно учитывать, если 0 или 1 не влияет
                # спама на русском языке 3, для писемь на английском языке 9 (гиперпараметр)
    
train_data = [  
    ['Купите новое чистящее средство', SPAM],   
    ['Купи мою новую книгу', SPAM],  
    ['Подари себе новый телефон', SPAM],
    ['Добро пожаловать и купите новый телевизор', SPAM],
    ['Привет давно не виделись', NOT_SPAM], 
    ['Довезем до аэропорта из пригорода всего за 399 рублей', SPAM], 
    ['Добро пожаловать в Мой Круг', NOT_SPAM],  
    ['Я все еще жду документы', NOT_SPAM],  
    ['Приглашаем на конференцию Data Science', NOT_SPAM],
    ['Потерял твой телефон напомни', NOT_SPAM],
    ['Порадуй своего питомца новым костюмом', SPAM],
] 

train_data2 = [  
    ["Развивай бизнес на дому с услугой 'Безлимитный Интернет'", NOT_SPAM],   
    ['Мы получили ваше сообщение о пропаже багажа и домашнего питомца в здании аэропорта. Конечно, нам жаль. Но что мы можем с этим сделать?', SPAM],  
    ['Перезвони по номеру +799999999 в течение 6 секунд и выиграй миллион рублей!', SPAM],
]

In [21]:
def dels(x):
    return x.replace(",","").replace("...","").replace(".","").replace("'","").replace(":","").replace(";","").\
    replace("!","").replace("?","").replace("(","").replace(")","").replace("[","").replace("]","").replace("{","").\
    replace("}","").replace("|","").replace("NUMBER","").replace("URL","")

def calculate_word_frequencies(body, label):
    if label == SPAM:
        SPAM_col[body] += 1
    else:
        NOT_SPAM_col[body] +=1

def train(train_data_):
    global SPAM_count, NOT_SPAM_count, P_All_SPAM
    
    SPAM_count += sum(1 for body, label in train_data_ if label == SPAM)
    NOT_SPAM_count += sum(1 for body, label in train_data_ if label == NOT_SPAM)
    
    if SPAM_count + NOT_SPAM_count > 0:
        P_All_SPAM = SPAM_count/(SPAM_count+NOT_SPAM_count)
        
    for body, label in train_data_:
        body = dels(body)
        for i in body.lower().split():
            if len(i)>=min_word_len:
                calculate_word_frequencies(i, label)        
        
def calculate_P_Bi_A(word, label):
    
    if SPAM_col[word] + NOT_SPAM_col[word] == 0:
        SPAM_col[word] +=1
        NOT_SPAM_col[word] +=1

    P_SPAM[word] = SPAM_col[word]/SPAM_word_count
    P_not_SPAM[word] = NOT_SPAM_col[word]/NOT_SPAM_word_count
    
    if label == SPAM:
        return P_SPAM[word]
    else:
        return P_not_SPAM[word]

    
def calculate_P_B_A(text, label):
    P = 1
    for i in text.lower().split():
        if len(i)>=min_word_len:
            P *= calculate_P_Bi_A(i,label)
    
    if label == SPAM:
        return P*P_All_SPAM
    else:
        return P*(1-P_All_SPAM)


def classify(email):
    global SPAM_word_count, NOT_SPAM_word_count
    SPAM_word_count = sum(SPAM_col.values())
    NOT_SPAM_word_count = sum(NOT_SPAM_col.values())
    email = dels(email)
    
    if calculate_P_B_A(email,SPAM) > calculate_P_B_A(email,NOT_SPAM):
        return SPAM_DICT[SPAM]
    else:
        return SPAM_DICT[NOT_SPAM]  
    
def classify_(email):
    global SPAM_word_count, NOT_SPAM_word_count
    SPAM_word_count = sum(SPAM_col.values())
    NOT_SPAM_word_count = sum(NOT_SPAM_col.values())
    email = dels(email)
    
    if calculate_P_B_A(email,SPAM) > calculate_P_B_A(email,NOT_SPAM):
        return SPAM
    else:
        return NOT_SPAM
    
def train_all():
    global df
   
    df = pd.read_csv('spam_or_not_spam.zip')
    df = df.dropna()
    df['label'] = df['label'].apply(lambda s: SPAM if (s == 1) else NOT_SPAM)
    df['email'] = df['email'].apply(lambda x: dels(x))
    train_data_en = df.values.tolist()
       
    train(train_data)
    train(train_data2)    
    train(train_data_en)

train_all()

In [22]:
df['label_pred'] = df.email.apply(lambda x: classify_(x))
print(confusion_matrix(df['label'],df['label_pred']))
print(f1_score(df['label'],df['label_pred']))

[[ 462   37]
 [  19 2481]]
0.9888401753686729


In [23]:
print(classify("Развивай бизнес на дому с услугой 'Безлимитный Интернет"))
print(classify("Мы получили ваше сообщение о пропаже багажа и домашнего питомца в здании аэропорта. Конечно, нам жаль. Но что мы можем с этим сделать?"))
print(classify("Перезвони по номеру +799999999 в течение 6 секунд и выиграй миллион рублей!"))

NOT SPAM
SPAM
SPAM


In [24]:
leter1 = "Hi, My name is Warren E. Buffett an American business magnate, investor and philanthropist. am the most successful investor in the world. I believe strongly in‘giving while living’ I had one idea that never changed in my mind? that you should use your wealth to help people and i have decided to give {$1,500,000.00} One Million Five Hundred Thousand United Dollars, to randomly selected individuals worldwide. On receipt of this email, you should count yourself as the lucky individual. Your email address was chosen online while searching at random. Kindly get back to me at your earliest convenience before i travel to japan for my treatment , so I know your email address is valid. Thank you for accepting our offer, we are indeed grateful You Can Google my name for more information: God bless you. Best Regard Mr.Warren E. Buffett Billionaire investor !"
leter2 = "Hi guys I want to build a website like REDACTED and I wanted to get your perspective of whether that site is good from the users' perspective before I go ahead and build something similar. I think that the design of the site is very modern and nice but I am not sure how people would react to a similar site? I look forward to your feedback. Many thanks!"
leter3 = "As a result of your application for the position of Data Engineer, I would like to invite you to attend an interview on May 30, at 9 a.m. at our office in Washington, DC. You will have an interview with the department manager, Moris Peterson. The interview will last about 45 minutes. If the date or time of the interview is inconvenient, please contact me by phone or email to arrange another appointment. We look forward to seeing you."
print(classify(leter1))
print(classify(leter2))
print(classify(leter3))

NOT SPAM
NOT SPAM
NOT SPAM
